## Haar Cascades

Based on the popular [Viola-Jones](https://link.springer.com/article/10.1023/B:VISI.0000013087.49260.fb) framework for object detection. They proposed using Haar filters (two-valued,  rectangular masks), efficiently computed using "integral" images (where each pixel value is replaced with the sum of all the pixel values above and to the left of it). Something like a 2D, discrete CDF in probability theory. They also proposed using a "cascade" of weak classifiers to improve performance.

OpenCV offers an implementation of this under the "CascadeClassifier" class, along with pretrained classifiers for face detection. 

In [37]:
import cv2

def detect_faces_haar(image_path, output_path): 
    # Path to preset face detection cascade file that come with OpenCV out of the box. Some other cascade files can be found at venv/Lib/site-packages/cv2/data
    face_detection_cascade_file = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(face_detection_cascade_file)     

    image = cv2.imread(image_path)
    cv2.imshow("Base image", image)
    faces_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Parameters for detectMultiScale taken from https://stackoverflow.com/questions/20801015/recommended-values-for-opencv-detectmultiscale-parameters 
    # along with some experiments on the sample image
    faces = face_cascade.detectMultiScale(faces_gray, scaleFactor = 1.2, minNeighbors = 4)      # these values can be changed as needed 

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imwrite(output_path, image)
    cv2.imshow("Detected faces (Haar Cascade)", image)
    print("Close windows or press key to exit.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
def detect_face_from_camera():

    face_detection_cascade_file = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(face_detection_cascade_file) 

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return "Error"

    print("Press 'q' to capture an image and 'e' to exit.")
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            cv2.imshow("Camera Feed", frame)
            key = cv2.waitKey(1)
            if key == ord("q"):
                gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray_img, scaleFactor = 1.2, minNeighbors = 4)      # these values can be changed as needed 

                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                cv2.imshow("Captured Image", frame)

            if key == ord("e"):
                break
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [38]:
detect_faces_haar("samples/face_detection/solvay_color.jpg", "samples/face_detection/solvay_detected_haar.jpg")

Close windows or press key to exit.


In [10]:
detect_face_from_camera()

Press 'q' to capture an image and 'e' to exit.


Works well on the sample image, but the camera feed performance is terrible.

## HOG + SVM

In [11]:
import cv2
import numpy as np

hog = cv2.HOGDescriptor()

image = cv2.imread("samples/face_detection/solvay_color.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
resized_image = cv2.resize(gray, (64, 128))  

hog_features = hog.compute(resized_image)

print("HOG Descriptor Shape:", hog_features.shape)
print("HOG Features:\n", hog_features)


HOG Descriptor Shape: (3780,)
HOG Features:
 [0.19350109 0.12668909 0.08527572 ... 0.12414946 0.07143334 0.12740421]
